Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic 

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        ) + tf.nn.l2_loss(weights) * .5e-2

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 36.002350
Minibatch accuracy: 5.5%
Validation accuracy: 8.2%
Minibatch loss at step 500: 6.010849
Minibatch accuracy: 80.5%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 2.524410
Minibatch accuracy: 78.9%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 1.136438
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.790615
Minibatch accuracy: 86.7%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 0.711671
Minibatch accuracy: 84.4%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 0.752991
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Test accuracy: 89.4%


In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    h1_wts = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    h1_b = tf.Variable(tf.zeros([1024]))
    h1_out = tf.nn.relu(tf.matmul(tf_train_dataset, h1_wts) + h1_b)
    
    h2_wts = tf.Variable(tf.truncated_normal([1024, num_labels]))
    h2_b = tf.Variable(tf.zeros([num_labels]))
    h2_out = tf.matmul(h1_out, h2_wts) + h2_b

    reg = tf.nn.l2_loss(h2_wts) + tf.nn.l2_loss(h1_wts)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(h2_out, tf_train_labels)
        ) + reg * .5e-3
    

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.4).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(h2_out)
    valid_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)
    test_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)

In [8]:
num_steps = 3501

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
                 )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 462.817139
Minibatch accuracy: 12.5%
Validation accuracy: 28.3%
Minibatch loss at step 500: 140.596634
Minibatch accuracy: 85.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 112.926666
Minibatch accuracy: 79.7%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 87.083160
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 69.319603
Minibatch accuracy: 86.7%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 56.674881
Minibatch accuracy: 92.2%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 46.120354
Minibatch accuracy: 85.9%
Validation accuracy: 84.3%
Minibatch loss at step 3500: 37.859039
Minibatch accuracy: 85.9%
Validation accuracy: 85.0%
Test accuracy: 91.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    h1_wts = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    h1_b = tf.Variable(tf.zeros([1024]))
    h1_out = tf.nn.relu(tf.matmul(tf_train_dataset, h1_wts) + h1_b)
    
    h2_wts = tf.Variable(tf.truncated_normal([1024, num_labels]))
    h2_b = tf.Variable(tf.zeros([num_labels]))
    h2_out = tf.matmul(h1_out, h2_wts) + h2_b

    reg = tf.nn.l2_loss(h2_wts) + tf.nn.l2_loss(h1_wts)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(h2_out, tf_train_labels)
        ) + reg * .5e-3
    

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.4).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(h2_out)
    valid_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)
    test_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)

In [10]:
num_steps = 501

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
#         offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = 0
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
                 )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 476.714478
Minibatch accuracy: 4.7%
Validation accuracy: 37.6%
Minibatch loss at step 500: 128.693085
Minibatch accuracy: 100.0%
Validation accuracy: 65.2%
Test accuracy: 71.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    h1_wts = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    h1_b = tf.Variable(tf.zeros([1024]))
    h1_out = tf.nn.relu(tf.matmul(tf_train_dataset, h1_wts) + h1_b)
    h1_out = tf.nn.dropout(h1_out, .5)
    
    h2_wts = tf.Variable(tf.truncated_normal([1024, num_labels]))
    h2_b = tf.Variable(tf.zeros([num_labels]))
    h2_out = tf.matmul(h1_out, h2_wts) + h2_b

    reg = tf.nn.l2_loss(h2_wts) + tf.nn.l2_loss(h1_wts)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(h2_out, tf_train_labels)
        ) + reg * .5e-3
    

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.4).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(h2_out)
    valid_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)
    test_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)

In [17]:
num_steps = 501

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = 0

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
                 )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 626.757385
Minibatch accuracy: 13.3%
Validation accuracy: 24.2%
Minibatch loss at step 500: 128.646912
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Test accuracy: 76.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [24]:
batch_size = 128
h1_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    h1_wts = tf.Variable(tf.truncated_normal([image_size * image_size, h1_size]))
    h1_b = tf.Variable(tf.zeros([h1_size]))
    h1_out = tf.nn.relu(tf.matmul(tf_train_dataset, h1_wts) + h1_b)
#     h1_out = tf.nn.dropout(h1_out, .5)
    
    h2_wts = tf.Variable(tf.truncated_normal([h1_size, num_labels]))
    h2_b = tf.Variable(tf.zeros([num_labels]))
    h2_out = tf.matmul(h1_out, h2_wts) + h2_b

    reg = tf.nn.l2_loss(h2_wts) + tf.nn.l2_loss(h1_wts)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(h2_out, tf_train_labels)
        ) + reg * .5e-3
    

    # Optimizer.
    global_step = tf.Variable(0)
    learn_rate = tf.train.exponential_decay(.4, global_step, 500, .96)
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(h2_out)
    valid_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)
    test_prediction = tf.nn.softmax(tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h1_wts) + h1_b),
            h2_wts) + h2_b)

In [25]:
num_steps = 3501

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
                 )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 601.082703
Minibatch accuracy: 14.1%
Validation accuracy: 38.6%
Minibatch loss at step 500: 135.836380
Minibatch accuracy: 85.2%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 108.237160
Minibatch accuracy: 82.0%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 89.952957
Minibatch accuracy: 87.5%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 74.180466
Minibatch accuracy: 86.7%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 62.871700
Minibatch accuracy: 84.4%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 52.714935
Minibatch accuracy: 85.2%
Validation accuracy: 84.0%
Minibatch loss at step 3500: 45.514194
Minibatch accuracy: 88.3%
Validation accuracy: 84.3%
Test accuracy: 91.4%
